# Exploartion

How does cycling in Stockholm change once autumn break ends and the city becomes darker, colder, and busier?
I built this data analysis project to explore how cyclist behavior shifts before and after höstlov, and what factors might explain those patterns.

## 1. Data Loading and Cleaning:

## 2. Exploratory Data Analysis (EDA)

### 2.1 Overall cycling level

#### Do cycling volumes decrease or increase in gerneral after höstlov?

So overall, cycling activity dropped by around 65 000 trips per week, or 14 %, after week 43.
Wilcoxon test: W = 0.000, p = 0.0039
→ This test is significant (p < 0.01).
It confirms that, across most years, week 45 has lower cycling counts than week 43.

#### How do cycling volumes on the same streets differ between the week before and the week after Höstlov?

### 2.2 Effect of Light on cycling level

#### Does street lighting influence wether people continue cycling after höstlov?

#### Do less people cycle in the dark hours after sunset?

### 2.3 cycling route changes

#### Do cyclist switch to different routes after höstlov?

### 2.4 Effect of Weather on cycling level

#### Do cycling volumes decrease because of the drop in temperature?

#### Do cycling volumes decrease because of the increase in rainfall?

### 2.5 Effect of traffic environment on cycling level

#### Do cyclists avoid shared bike-pedestrian paths routes, particularly at night due to less visibility?

#### Do cyclists avoid Side-by-side opposing cycle lanes, particularly at night due to less visibility?

#### Do bidirectional cycle lanes experience lower cyclist volumes?

### Summary

text 
text text

## Primary Conclusion
text
text
text

# Presentation ("Flipping the Pyramid")

## Executive Summary
text
text text

## 1. Key finding

## 2. Key finding

## 3. Key finding

## 4. Key finding

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import requests
import pytz
from astral import LocationInfo
from astral.sun import sun
from scipy import stats

sns.set(style="whitegrid")


In [2]:
# Load core datasets

df_dst = pd.read_csv('stockholm_dst_end_2015_2025.csv', encoding='utf-8')
df_cykel = pd.read_csv('td_cykel_15_min_2015-2024.csv', encoding='ascii', sep=';')
df_platser = pd.read_csv('platser_cykel_2015-2024.csv', encoding='utf-8', sep=';')
df_hostlov = pd.read_csv('hostlov_2015_2023.csv')


In [3]:
# Core cleaning and feature engineering for all later analyses

# Basic tidy-up
df_cykel = df_cykel.drop(columns=['direction'])

# Convert hostlov dates
df_hostlov['Start'] = pd.to_datetime(df_hostlov['Start'])
df_hostlov['End'] = pd.to_datetime(df_hostlov['End'])

# DST date parsing
df_dst['Date'] = pd.to_datetime(df_dst['Date'], format='%d-%b-%y', errors='coerce')

# Cycling timestamps: UTC -> Stockholm local
df_cykel['timestamp_utc'] = pd.to_datetime(df_cykel['tidsstampel'], utc=True, errors='coerce')
df_cykel = df_cykel.dropna(subset=['timestamp_utc'])
df_cykel['timestamp_local'] = df_cykel['timestamp_utc'].dt.tz_convert('Europe/Stockholm')

# Numeric fields
df_cykel['matplats_id'] = pd.to_numeric(df_cykel['matplats_id'], errors='coerce').astype('Int64')
df_cykel['antal'] = pd.to_numeric(df_cykel['antal'], errors='coerce').astype('Int64')

# Time parts
df_cykel['date_local'] = df_cykel['timestamp_local'].dt.date
df_cykel['hour_local'] = df_cykel['timestamp_local'].dt.hour
df_cykel['dow_local'] = df_cykel['timestamp_local'].dt.day_name()
df_cykel['year'] = df_cykel['timestamp_local'].dt.year

# Remove skewed / problematic data
# outlier: matplats_id 6633 in 2015
df_cykel = df_cykel[~((df_cykel['matplats_id'] == 6633) & (df_cykel['year'] == 2015))].copy()

# Drop incomplete years 2024-2025
df_cykel = df_cykel[~df_cykel['year'].isin([2024, 2025])].copy()

# ISO year/week for all rows
iso_all = df_cykel['timestamp_local'].dt.isocalendar()
df_cykel['iso_year'] = iso_all.year.astype(int)
df_cykel['iso_week'] = iso_all.week.astype(int)


In [4]:
# Helper functions used throughout the analysis

def add_week_aggregate(df, weeks=(43, 45)):
    """Return df aggregated by matplats_id, iso_year, iso_week for selected weeks."""
    temp = df[df['iso_week'].isin(weeks)].copy()
    agg = (
        temp.groupby(['matplats_id', 'iso_year', 'iso_week'], as_index=False)['antal']
        .sum()
        .rename(columns={'antal': 'total_cyclists'})
    )
    return agg


def pivot_week_change(weekly_df, week_a=43, week_b=45):
    """Pivot week-wise totals and compute percentage change from week_a to week_b."""
    pivot = (
        weekly_df
        .pivot(index=['matplats_id', 'iso_year'], columns='iso_week', values='total_cyclists')
        .rename(columns={week_a: 'week' + str(week_a), week_b: 'week' + str(week_b)})
        .reset_index()
    )
    col_a = 'week' + str(week_a)
    col_b = 'week' + str(week_b)
    valid = pivot.dropna(subset=[col_a, col_b]).copy()
    valid['pct_change'] = (valid[col_b] - valid[col_a]) / valid[col_a] * 100
    return pivot, valid


def mark_hostlov_period(df, hostlov_df):
    """Add a boolean column in_hostlov based on hostlov start/end per year."""
    df = df.copy()
    df['in_hostlov'] = False
    for _, row in hostlov_df.iterrows():
        mask = (df['timestamp_local'] >= row['Start']) & (df['timestamp_local'] <= row['End'])
        df.loc[mask, 'in_hostlov'] = True
    return df


In [5]:
# Week 43 vs 45 analysis shared for multiple plots

weekly_totals = add_week_aggregate(df_cykel, weeks=(43, 45))
pivot_all, pivot_valid = pivot_week_change(weekly_totals, week_a=43, week_b=45)

years_per_site = (
    pivot_valid.groupby('matplats_id')['iso_year']
    .nunique()
    .reset_index(name='n_years')
)

all_sites = df_cykel[['matplats_id']].drop_duplicates().sort_values('matplats_id')
years_per_site_full = all_sites.merge(years_per_site, on='matplats_id', how='left')
years_per_site_full['n_years'] = years_per_site_full['n_years'].fillna(0).astype(int)

few_year_sites = years_per_site_full[years_per_site_full['n_years'] <= 2]

print('Matplats with 2 or fewer usable years for week 43->45 change:')
print(few_year_sites.sort_values(['n_years', 'matplats_id']))

# Detailed summary for those sites
detailed_all = (
    pivot_all.merge(few_year_sites[['matplats_id']], on='matplats_id', how='inner')
    .sort_values(['matplats_id', 'iso_year'])
)

detailed_all = detailed_all.merge(
    pivot_valid[['matplats_id', 'iso_year', 'pct_change']],
    on=['matplats_id', 'iso_year'],
    how='left'
)


Matplats with 2 or fewer usable years for week 43->45 change:
     matplats_id  n_years
1           6326        0
2           6327        0
3           6337        0
4           6345        0
5           6346        0
..           ...      ...
69          6627        1
70          6628        1
130         8293        1
85          6647        2
131         8294        2

[84 rows x 2 columns]


In [6]:
print(df_cykel.head(5))

   matplats_id           tidsstampel  antal  year  month  dag  \
0         6265  2015-01-01T01:45:00Z      1  2015      1    1   
1         6265  2015-01-01T02:00:00Z      1  2015      1    1   
2         6265  2015-01-01T02:45:00Z      1  2015      1    1   
3         6265  2015-01-01T03:45:00Z      1  2015      1    1   
4         6265  2015-01-01T06:15:00Z      1  2015      1    1   

              timestamp_utc           timestamp_local  date_local  hour_local  \
0 2015-01-01 01:45:00+00:00 2015-01-01 02:45:00+01:00  2015-01-01           2   
1 2015-01-01 02:00:00+00:00 2015-01-01 03:00:00+01:00  2015-01-01           3   
2 2015-01-01 02:45:00+00:00 2015-01-01 03:45:00+01:00  2015-01-01           3   
3 2015-01-01 03:45:00+00:00 2015-01-01 04:45:00+01:00  2015-01-01           4   
4 2015-01-01 06:15:00+00:00 2015-01-01 07:15:00+01:00  2015-01-01           7   

  dow_local  iso_year  iso_week  
0  Thursday      2015         1  
1  Thursday      2015         1  
2  Thursday      201

In [7]:
print(df_dst.head(5))

   Year       Date Day of Week  ISO Week  from      to
0  2015 2015-10-25      Sunday        43  3:00   02:00
1  2016 2016-10-30      Sunday        43  3:00   02:00
2  2017 2017-10-29      Sunday        43  3:00   02:00
3  2018 2018-10-28      Sunday        43  3:00   02:00
4  2019 2019-10-27      Sunday        43  3:00   02:00


In [8]:
print(df_platser.head(5))

   MätplatsID         Mätplatsnamn  \
0        6522             C0620X01   
1        6525             C0619X01   
2        6646  C 0306 Munkbroleden   
3        6345             C0439X01   
4        6346             C0439X02   

                                 MätplatsBeskrivning              Sträcka  \
0                             Ulvsundavägen, ID 126V        Ulvsundavägen   
1                                Brommaplan, ID 126N  Drottningholmsvägen   
2                       Cykelbana längs Munkbroleden         Munkbroleden   
3         NYNÄSVÄGEN del SÖNDAGSVÄGEN - MÅNDAGSVÄGEN  NYNÄSVÄGEN cykelväg   
4  NYNÄSVÄGEN del SKÖNSTAHOLMSVÄGEN - LESJÖFORSGRÄND           Nynäsvägen   

          RiktplatsA      RiktplatsB                                   R1  \
0    Lillsjönäsvägen    Ulvsundaplan      Lillsjönäsvägen => Ulvsundaplan   
1        Riksbyvägen      Brommaplan            Riksbyvägen => Brommaplan   
2         Kåkbrinken    Munkbrogatan           Kåkbrinken => Munkbrogatan   


In [9]:
df_cykel = df_cykel.drop(columns=['direction'])

KeyError: "['direction'] not found in axis"

In [ ]:
print(df_cykel.columns.tolist())

In [ ]:
zero_count = (df_cykel['antal'] == 0).sum()
print("Zero values in antal:", zero_count)

In [ ]:
print(df_dst.dtypes)

In [ ]:
print(df_cykel.dtypes)

In [ ]:
print(df_platser.dtypes)

In [ ]:
print(df_hostlov.dtypes)

In [ ]:
# convert start and end columns into datetime objects
df_hostlov['Start'] = pd.to_datetime(df_hostlov['Start'])
df_hostlov['End'] = pd.to_datetime(df_hostlov['End'])

In [ ]:
print(df_hostlov.dtypes)

In [ ]:
# Extract year directly from the already-cleaned timestamp_local
df_cykel['year'] = df_cykel['timestamp_local'].dt.year

# Group by year and count total cyclists and number of measurements
year_summary = (
    df_cykel.groupby('year', as_index=False)
    .agg(
        total_cyclists=('antal', 'sum'),
        measurements=('antal', 'count')
    )
    .sort_values('year')
)

print(year_summary)

In [ ]:
# drop skewed years (2024 and 2025)
df_cykel = df_cykel[~df_cykel['year'].isin([2024, 2025])].copy()

print("Remaining years in df_cykel:", sorted(df_cykel['year'].dropna().unique()))

In [ ]:
print(df_cykel.columns.tolist())

In [ ]:
# --- 10. Detailed summary only for sites with ≤2 years (including years where change could NOT be calculated) ---

# 1) Merge pivot_all with few_year_sites to keep only matplats_id with n_years <= 2
detailed_all = (
    pivot_all.merge(few_year_sites[["matplats_id"]], on="matplats_id", how="inner")
             .sort_values(["matplats_id", "year"])
)

# 2) Bring pct_change from pivot_valid (only for rows where both weeks exist)
detailed_all = detailed_all.merge(
    pivot_valid[["matplats_id", "year", "pct_change"]],
    on=["matplats_id", "year"],
    how="left"
)

# Now detailed_all has:
# - matplats_id, year
# - week43, week45 (can be  if missing)
# - pct_change ( where either week was missing)

print("\nDetailed summary (week43, week45, pct_change) for sites with ≤2 years (including non-calculable years):")
print(
    detailed_all[["matplats_id", "year", "week43", "week45", "pct_change"]]
    .to_string(
        index=False,
        formatters={
            "week43": lambda x: "" if pd.isna(x) else "{:,.0f}".format(x),
            "week45": lambda x: "" if pd.isna(x) else "{:,.0f}".format(x),
            "pct_change": lambda x: "" if pd.isna(x) else "{:.1f}%".format(x)
        }
    )
)

In [ ]:
# --- 11. List of matplats_id to exclude (sites with ≤2 valid years, including 0) ---
exclude_ids = sorted(few_year_sites["matplats_id"].unique().tolist())

print("\nmatplats_id to EXCLUDE in future plots (≤2 years of usable week 43→45 change, including 0-year sites):")
print(exclude_ids)

In [ ]:
# 6627, 6628, 6635, 6647, 8293, 8294 not enough data to analyze week 43 and 45
# 6647, 6648, 6729 mätningsplats works on battery and had huge outagges and that led to unrealistic cyling volumes

ids_to_remove = [6647, 6648, 6729, 6627, 6628, 6635, 6647, 8293, 8294]

# totals BEFORE removing IDs
total_antal_before = df_cykel['antal'].sum()
total_rows_before = df_cykel.shape[0]

# how much cyclist volume would be removed
removed_antal = df_cykel[df_cykel['matplats_id'].isin(ids_to_remove)]['antal'].sum()
removed_rows = df_cykel[df_cykel['matplats_id'].isin(ids_to_remove)].shape[0]

# reduction in %
removed_pct = (removed_antal / total_antal_before) * 100

print(f" Total cyclists BEFORE removal: {total_antal_before:,}")
print(f" Cyclists removed (antal): {removed_antal:,}")
print(f" Percentage removed: {removed_pct:.2f}%")

In [ ]:
# 6627, 6628, 6635, 6647, 8293, 8294 not enough data to analyze week 43 and 45
# 6647, 6648, 6729 mätningsplats works on battery and had huge outagges and that led to unrealistic cyling volumes

# IDs to remove completely
ids_to_remove = [6647, 6648, 6729, 6627, 6628, 6635, 6647, 8293, 8294]

# Remove from df_cykel
df_cykel = df_cykel[~df_cykel['matplats_id'].isin(ids_to_remove)].copy()

# Remove from df_platser
df_platser = df_platser[~df_platser['MätplatsID'].isin(ids_to_remove)].copy()

print("✔️ Unwanted IDs removed from df_cykel and df_platser")

In [ ]:
from IPython.display import Image, display

display(Image(filename="Magelungsvägen.png"))

# no cyclepath lighting for more than 400 meters
# the highway lighting is not sufficient for safe cycling in autumn/winter evenings

In [ ]:
from IPython.display import Image, display

display(Image(filename="Örbyleden.png"))

# no cyclepath lighting for more than 500 meters
# the highway lighting is not sufficient for safe cycling in autumn/winter evenings

In [ ]:
from IPython.display import Image, display

display(Image(filename="Hjorthagen.png"))

# the distances between lighting points are too long some distances are more than 60 meters
# they create dark zones along the cycle path making it unsafe for cyclists in autumn/winter evenings

In [ ]:
# I choose the year I want to look at
YEAR = 2023

# I set the timezone and location information for Stockholm
TZ = 'Europe/Stockholm'
LAT = 59.3293
LON = 18.0686
CITY_NAME = "Stockholm"

# I set the start and end dates for October and November of the chosen year
start_date = pd.Timestamp(f'{YEAR}-10-01')
end_date = pd.Timestamp(f'{YEAR}-11-30 23:59:59')

# I make sure timestamp_local has no timezone (so they match)
df_cykel['timestamp_local_naive'] = df_cykel['timestamp_local'].dt.tz_localize(None)

# I keep only the data between these two dates
mask = (df_cykel['timestamp_local_naive'] >= start_date) & (df_cykel['timestamp_local_naive'] <= end_date)
df_sel = df_cykel.loc[mask, ['timestamp_local_naive', 'antal']].copy()

# I add up the number of cyclists per day
daily = (
    df_sel
    .set_index('timestamp_local_naive')
    .resample('D')['antal']
    .sum()
    .to_frame(name='daily_total')
    .reset_index()
)

# I create some helper columns for easier plotting and display
daily['date_local'] = daily['timestamp_local_naive']
daily['date_only'] = daily['timestamp_local_naive'].dt.date
daily['date_str'] = daily['timestamp_local_naive'].dt.strftime('%Y-%m-%d')

# I get the location info for Stockholm so I can calculate sunrise and sunset
location = LocationInfo(name=CITY_NAME, region="Sweden", timezone=TZ, latitude=LAT, longitude=LON)
tzinfo = pytz.timezone(TZ)

# I prepare empty lists to store sunrise and sunset information
sunrise_list, sunset_list = [], []
sunrise_str, sunset_str = [], []
sunrise_hour, sunset_hour = [], []

# I go through each date and calculate the sunrise and sunset times
for idx, row in daily.iterrows():
    date = row['date_only']
    try:
        s = sun(location.observer, date=date, tzinfo=tzinfo)
        sr, ss = s['sunrise'], s['sunset']
    except Exception:
        sr = ss = None

    if (sr is None) or (ss is None):
        sunrise_list.append(pd.NaT)
        sunset_list.append(pd.NaT)
        sunrise_str.append(None)
        sunset_str.append(None)
        sunrise_hour.append(np.nan)
        sunset_hour.append(np.nan)
        continue

    # I convert the times to Stockholm local and remove the timezone
    sr_naive = sr.astimezone(tzinfo).replace(tzinfo=None)
    ss_naive = ss.astimezone(tzinfo).replace(tzinfo=None)

    sunrise_list.append(sr_naive)
    sunset_list.append(ss_naive)
    sunrise_str.append(sr_naive.strftime('%H:%M:%S'))
    sunset_str.append(ss_naive.strftime('%H:%M:%S'))
    sunrise_hour.append(sr_naive.hour + sr_naive.minute / 60)
    sunset_hour.append(ss_naive.hour + ss_naive.minute / 60)

# I add all these values back into my dataframe
daily['sunrise'] = sunrise_list
daily['sunset'] = sunset_list
daily['sunrise_str'] = sunrise_str
daily['sunset_str'] = sunset_str
daily['sunrise_hour'] = sunrise_hour
daily['sunset_hour'] = sunset_hour

# I create an interactive chart with daily cycling totals and sunrise/sunset times
fig = go.Figure()

# I add the bar chart for daily cycling totals
fig.add_trace(
    go.Bar(
        x=daily['date_local'],
        y=daily['daily_total'],
        name='Cyclists per day',
        marker_color='steelblue',
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Total cyclists: %{y}<extra></extra>'
    )
)

# I add the line for sunrise times
fig.add_trace(
    go.Scatter(
        x=daily['date_local'],
        y=daily['sunrise_hour'],
        name='Sunrise (hour)',
        mode='lines+markers',
        line=dict(color='orange', dash='dash'),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Sunrise: %{customdata[0]}<extra></extra>',
        customdata=daily[['sunrise_str']].values,
        yaxis='y2'
    )
)

# I add the line for sunset times
fig.add_trace(
    go.Scatter(
        x=daily['date_local'],
        y=daily['sunset_hour'],
        name='Sunset (hour)',
        mode='lines+markers',
        line=dict(color='red', dash='dash'),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Sunset: %{customdata[0]}<extra></extra>',
        customdata=daily[['sunset_str']].values,
        yaxis='y2'
    )
)

# I adjust the layout of the chart
fig.update_layout(
    title=f'Daily Cycling Volumes and Sunrise/Sunset Times — Oct & Nov {YEAR} (Stockholm)',
    xaxis_title='Date',
    yaxis=dict(title='Total cyclists per day', side='left'),
    yaxis2=dict(title='Hour of day (sunrise/sunset)', overlaying='y', side='right', range=[0, 24]),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
    hovermode='x unified',
    xaxis_tickformat='%Y-%m-%d',
    xaxis_tickangle=-45,
    margin=dict(l=60, r=60, t=80, b=120)
)

# I show the chart
fig.show()

In [ ]:
# The biggest decline in cycling happens after the clocks change at the end of October, when: darkness arrives earlier (16:00–17:00)
# evening commutes fall into dark hours

In [ ]:
# I make sure the timestamp column is in datetime format
df_cykel['timestamp_local'] = pd.to_datetime(df_cykel['timestamp_local'])

# I only keep data from 2015 to 2023
mask = (df_cykel['timestamp_local'].dt.year >= 2015) & (df_cykel['timestamp_local'].dt.year <= 2023)
df_cykel = df_cykel.loc[mask]

# I create new columns for date and hour
df_cykel['date'] = df_cykel['timestamp_local'].dt.date
df_cykel['hour'] = df_cykel['timestamp_local'].dt.hour

# I count how many cyclists there were per date and hour
heatmap_df = df_cykel.groupby(['date', 'hour'], as_index=False)['antal'].sum()

# I create a heatmap that shows when most cyclists were active
fig = px.density_heatmap(
    heatmap_df,
    x='hour',
    y='date',
    z='antal',
    color_continuous_scale='YlOrRd',
    labels={'hour': 'Hour of Day', 'date': 'Date', 'antal': 'Number of Cyclists'},
    title='Cycling Traffic Heatmap — 2015 to 2023'
)

# I make sure every hour appears on the x-axis
fig.update_xaxes(dtick=1)

# I keep the date order on the y-axis
fig.update_yaxes(type='category')

# I show the heatmap
fig.show()

In [ ]:
# I make sure my timestamp column is in the right format
df_cykel['timestamp_local'] = pd.to_datetime(df_cykel['timestamp_local'], errors='coerce')

# I remove rows where the timestamp is missing
df_cykel = df_cykel.dropna(subset=['timestamp_local'])

# I extract useful time information
df_cykel['date'] = df_cykel['timestamp_local'].dt.date
df_cykel['hour'] = df_cykel['timestamp_local'].dt.hour
iso = df_cykel['timestamp_local'].dt.isocalendar()
df_cykel['year'] = iso.year.astype(int)
df_cykel['week'] = iso.week.astype(int)

# I keep only data from week 43 between 2015 and 2023
df_w43 = df_cykel[
    (df_cykel['week'] == 43) &
    (df_cykel['year'].between(2015, 2023))
]

# I check that I have data for this period
if df_w43.empty:
    raise ValueError("No data available for week 43 (2015–2023) in this dataset.")

# I count how many cyclists there were each hour of each day
hourly_day = (
    df_w43.groupby(['date', 'hour'], as_index=False)['antal']
    .sum()
)

# I create a box plot to see how cycling changes by hour
fig = px.box(
    hourly_day,
    x='hour',
    y='antal',
    points='all',  # I show all individual data points
    labels={
        'hour': 'Hour of Day (local time)',
        'antal': 'Cyclists per Hour',
    },
    title='Cycling Traffic by Hour — Week 43 (2015–2023)'
)

# I make the chart easier to read
fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis_title='Cyclist Count per Hour',
    hovermode='x unified'
)

# I show the plot
fig.show()

In [ ]:
# I make sure my timestamp column is in the right format
df_cykel['timestamp_local'] = pd.to_datetime(df_cykel['timestamp_local'], errors='coerce')

# I remove rows where the timestamp is missing
df_cykel = df_cykel.dropna(subset=['timestamp_local'])

# I extract useful time information
df_cykel['date'] = df_cykel['timestamp_local'].dt.date
df_cykel['hour'] = df_cykel['timestamp_local'].dt.hour
iso = df_cykel['timestamp_local'].dt.isocalendar()
df_cykel['year'] = iso.year.astype(int)
df_cykel['week'] = iso.week.astype(int)

# I keep only data from week 45 between 2015 and 2023
df_w45 = df_cykel[
    (df_cykel['week'] == 45) &
    (df_cykel['year'].between(2015, 2023))
]

# I check that I have data for this period
if df_w45.empty:
    raise ValueError("No data available for week 45 (2015–2023) in this dataset.")

# I count how many cyclists there were each hour of each day
hourly_day = (
    df_w45.groupby(['date', 'hour'], as_index=False)['antal']
    .sum()
)

# I create a box plot to see how cycling changes by hour
fig = px.box(
    hourly_day,
    x='hour',
    y='antal',
    points='all',  # I show all individual data points
    labels={
        'hour': 'Hour of Day (local time)',
        'antal': 'Cyclists per Hour',
    },
    title='Cycling Traffic by Hour — Week 45 (2015–2023)'
)

# I make the chart easier to read
fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis_title='Cyclist Count per Hour',
    hovermode='x unified'
)

# I show the plot
fig.show()

In [ ]:
# I create a period column based on the hour of the day
def period_of_day(hour):
    if 6 <= hour <= 9:
        return 'Morning (06–09)'
    elif 10 <= hour <= 15:
        return 'Midday (10–15)'
    elif 16 <= hour <= 20:
        return 'Evening (16–20)'
    else:
        return 'Night (21–05)'

# I make sure the hour column exists
df_cykel['hour'] = df_cykel['timestamp_local'].dt.hour

# I add the period column
df_cykel['period'] = df_cykel['hour'].apply(period_of_day)

# I group the data by path and time of day to see which periods exist
period_presence = (
    df_cykel
    .groupby(['matplats_id', 'period'])['antal']
    .count()              # I count how many measurements exist
    .reset_index()        # I turn the grouped data back into a dataframe
    .pivot(               # I create columns for each period
        index='matplats_id',
        columns='period',
        values='antal'
    )
)

# I show the first 15 paths to check which periods have data
print(period_presence.head(15))

In [ ]:
# I check which time periods exist for each path
period_counts = (
    df_cykel
    .groupby(['matplats_id', 'period'])['antal']
    .count()
    .reset_index()
    .pivot(index='matplats_id', columns='period', values='antal')
)

print(period_counts.head(15))

In [ ]:
#e.g. The +397,984 value for Midday (10–15) means that, when you total up all the data from 2015–2023, there were about 397,984 fewer midday cycling trips in week 45 compared to week 43.
# After week 43, people cycled much less in the middle of the day and in the evenings, but morning cycling actually went up a bit.

In [ ]:
# Cycling in Stockholm is strongly daylight-dependent.
# The darker it gets, the fewer people continue cycling — especially in the evening.

In [ ]:
# no real shift in cyle routs more a reduction over all in cyclist amount

In [ ]:
#Open-Meteo Historical Weather API
# # Weather Data: Temperature, Precipitation, and Daylight (Stockholm, 2015–2023)

import requests
from tqdm.notebook import tqdm  # progress bar for multiple years

# Coordinates for Stockholm
lat, lon = 59.3293, 18.0686

# Years to fetch
years = range(2015, 2024)

# Container for all years
frames = []

print("Fetching weather data from Open-Meteo...")

for year in tqdm(years):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&daily=temperature_2m_mean,precipitation_sum,daylight_duration"
        f"&timezone=Europe/Stockholm"
    )

    resp = requests.get(url, timeout=30)
    resp.raise_for_status()  # raise an error if something fails
    data = resp.json()

    if "daily" not in data:
        print(f"⚠️ No data for {year}")
        continue

    df_year = pd.DataFrame(data["daily"])
    df_year["date"] = pd.to_datetime(df_year["time"])
    frames.append(df_year)

# Combine all years
df_weather = pd.concat(frames, ignore_index=True)

# Clean and rename columns
df_weather = df_weather.rename(
    columns={
        "temperature_2m_mean": "temp_mean",
        "precipitation_sum": "precip_sum",
        "daylight_duration": "daylight_seconds",
    }
)

# Convert daylight to hours
df_weather["daylight_hours"] = df_weather["daylight_seconds"] / 3600

# Add year and ISO week
df_weather["year"] = df_weather["date"].dt.year
df_weather["week"] = df_weather["date"].dt.isocalendar().week

# Show a preview
print(df_weather.head())
print(df_weather.tail())

# Weekly aggregation
weekly_weather = (
    df_weather.groupby(["year", "week"], as_index=False)
    .agg(
        temp_mean=("temp_mean", "mean"),
        precip_total=("precip_sum", "sum"),
        daylight_mean=("daylight_hours", "mean")
    )
)

print("\n✅ Finished building weekly weather dataset!")
print(weekly_weather.head())

In [ ]:
# The three opposing-direction cycling paths (IDs 6686, 6671, and 6680) account for 7,213 lost cyclists, representing 19.3% of the entire cyclist volume drop observed across the Top-20 highest-decline cycling paths in Stockholm.
# Interpretation:
#
# Opposing-direction cycling lanes make up 3 out of the 20 Top-20 paths
# → that is 15% of all high-drop routes.
#
# However, these 3 paths account for 19.3% of the total cyclist drop
# → almost 20% of all lost cyclists.
#
# This means their impact (~20%) is larger than their representation (15%),
# indicating that opposing-direction lanes contribute more strongly to the
# overall decline than would be expected based on their number alone.